<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Camilo Carvajal Reyes
- Nombre de alumno 2: Andrés Gonzalez Fuentes


### **Link de repositorio de GitHub:** `https://github.com/camilocarvajalreyes/laboratorios-MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [28]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [29]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [30]:
def series(customer,column,df_in=df_retail):
    """Función auxiliar que retorna una serie asociada a las ocurrencias de un clienete particular"""
    return df_in[df_in['Customer ID'] == customer][column]

def sub_df(customer,df_in=df_retail):
    """Función auxiliar que retorna una dataframe asociado a las ocurrencias de un clienete particular"""
    return df_in[df_in['Customer ID'] == customer]

def custom_features(dataframe_in):
    df = dataframe_in[['Customer ID']].copy(deep=True)
    df = df.drop_duplicates()
    # variables útiles
    t_final = df_retail['InvoiceDate'].max() + pd.DateOffset(1)
    # LRMPF
    delta_fecha = lambda row: (series(row['Customer ID'],'InvoiceDate').max() - series(row['Customer ID'],'InvoiceDate').min()).days
    df['Length'] = df.apply(delta_fecha,axis=1)

    n = 4
    recency = lambda row: ((series(row['Customer ID'],'InvoiceDate').drop_duplicates().nlargest(n)).apply(lambda date: (t_final-date).days)).mean()
    df['Recency'] = df.apply(lambda val: round(recency(val)),axis=1)

    monetary = lambda row: sub_df(row['Customer ID']).groupby(['InvoiceDate']).Price.sum().mean()
    df['Monetary'] = df.apply(monetary,axis=1)

    frequency = lambda row: len(series(row['Customer ID'],'InvoiceDate').drop_duplicates())
    df['Frequency'] = df.apply(frequency,axis=1)

    periodicity = lambda row: ((series(row['Customer ID'],'InvoiceDate').drop_duplicates()).diff().apply(lambda val: val if pd.isnull(val) else val.days)).std()
    df['Periodicity'] = df.apply(lambda val: 0 if pd.isnull(periodicity(val)) else round(periodicity(val)),axis=1)

    return df

In [31]:
# df_LRMPF = custom_features(df_retail[df_retail['Customer ID'].isin([12346.0,12347.0,12349.0,12352.0,12356.0])])
df_LRMPF = custom_features(df_retail)
# print('Presenta un total de {} filas con Customer IDs duplicados.'.format(df_LRMPF.duplicated('Customer ID').sum()))

In [32]:
df_LRMPF[df_LRMPF['Customer ID'].isin([12346.0,12347.0,12349.0,12352.0,12356.0])].sort_values('Customer ID')

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
20724,12346.0,196,275,18.760000,11,37
326853,12347.0,37,22,81.475000,2,0
131582,12349.0,181,158,291.780000,3,102
349604,12352.0,16,20,27.300000,2,0
293951,12356.0,44,35,86.476667,3,13


**Visualización**

In [33]:
fig1 = px.histogram(
    df_LRMPF['Length']
)
fig2 = px.histogram(
    df_LRMPF['Recency']
)
fig3 = px.histogram(
    df_LRMPF['Frequency']
)
fig4 = px.histogram(
    df_LRMPF['Monetary']
)
fig5 = px.histogram(
    df_LRMPF['Periodicity']
)

fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

**Observaciones**

    Length: Por lejos la barra más poblada es aquella que contiene hasta un valor de Length 9, lo cual debe explicarce gracias a la gran cantidad de gente que solo visito el sitio una vez (ver valor frequency). Es interesante notar que se observa un leve crecimiento en la frecuencia de personas que tienen una Recency de 350, i.e., casi equivalente al periodo de observación completo. Una posible explicación para esto es que los compradores usuales cubren casi todo el intervalo temporal presente en el dataset.

    Recency: La mayoría de los clientes se aglutinan en torno a valores del orden de 50 de recency, disminuyendo la densidad hasta los 350. Los valores cercanos algún valor m nos dice que para aquellos clientes, o bien se realizaron las últimas n compras m días antes del fin de la observación, o bien realizaron varias, relativamente esparcidas pero cuya media es m. Acá consideramos las últimas 4 compras, lo cual puede cargar la distribución hacia valores recientes comparado a haber usado valores más altos. Aquellos valores que están del orden de 360 días corresponden a clientes que probablemente compraron pocas veces y bien al principio del periodo de observación.

    Frequency: se nota que gran parte de los clientes ha realizado pocas visitas al sitio, i.e., que no son compradores que compraron una cantidad excesiva de veces durante el periodo de observación.

    Monetary: con una distribución que parece correlacionar con Frequency, Monetary tiene un peak entre 30 y 50 unidades monetarias. Naturalmente se disminuye rapidamente, sin embargo vale notar que existen valores que van suben mucho, i.e., clientes que gastaron mucho dinero.

    Periodicidad: Este histograma muestra un fuerte peak en valores bajos, lo cual debe explicarse ya que muchos clientes solo visitaron el sitio una o dos veces, con lo cual esa desviación estándar equivale a cero. Para clientes con mayor frecuencia, se observan varios valores entre 10 y 30.

    En todos los casos, se observa la importancia de escalar los datos para poder llevar a cabo una exploración consistente.

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [34]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.xmin = X.min()
        self.xmax = X.max()
        return self
    def transform(self, X):
        return (X - self.xmin)/(self.xmax - self.xmin)

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [35]:
np.random.seed(42)

scaling_minmax = Pipeline([('scaler',MinMax())])

minmax_transformer = ColumnTransformer(
    transformers=[
        ('MinMaxScaler',MinMax(), ['Length','Recency','Frequency','Monetary','Periodicity'])
])

lrmfp_tsne_pipeline = Pipeline([
    # aplicar transformaciones para custom features
    ('LRMFP',FunctionTransformer(custom_features)),
    ('MinMaxScaler',minmax_transformer),
    ('TSNE',TSNE())
])

In [36]:
%%capture
X_tsne = lrmfp_tsne_pipeline.fit_transform(df_retail)

In [37]:
fig = px.scatter(
    df_LRMPF, x=X_tsne[:,0], y=X_tsne[:,1], title='Visualización de atributos LRMPF con t-SNE',
    hover_name='Customer ID', hover_data=['Length','Recency','Frequency','Monetary','Periodicity']
)

fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [38]:
inertias = []
for n in range(1,21):
    lrmfp_kmeans_elbow_pipeline = Pipeline([
        # aplicar transformaciones para custom features
        ('LRMFP',FunctionTransformer(custom_features)),
        ('MinMaxScaler',minmax_transformer),
        ('KMEANS',KMeans(n_clusters=n))
    ])
    lrmfp_kmeans_elbow_pipeline.fit_transform(df_retail)
    inertias.append([n, lrmfp_kmeans_elbow_pipeline.named_steps['KMEANS'].inertia_])

In [39]:
inertias = pd.DataFrame(inertias, columns=["n° clusters", "inertia"])
inertias

,n° clusters,inertia
0,1,900.666077
1,2,423.368443
2,3,259.774660
3,4,196.919604
4,5,160.915265
5,6,140.098983
6,7,120.886381
7,8,105.870788
8,9,96.968803
9,10,89.645244


In [40]:
px.line(
    inertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
    markers=True
)

```
A raíz del gráfico construido con el método del codo, el número de clústers elegido para los siguientes pasos será el k=4, pues es a partir desde este valor que los valores de la 'inertia' comienzan a descender a un 'ritmo' considerablemente más bajo que los pasos anteriores.
```

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [41]:
n_clusters = 4

lrmfp_kmeans_pipeline = Pipeline([
        # aplicar transformaciones para custom features
        ('LRMFP',FunctionTransformer(custom_features)),
        ('MinMaxScaler',minmax_transformer),
        ('KMEANS',KMeans(n_clusters=n_clusters)) # Falta elegir el n óptimo
    ])

In [42]:
X_kmeans = lrmfp_kmeans_pipeline.fit_transform(df_retail)

df_lrmfp = lrmfp_kmeans_pipeline.named_steps['LRMFP'].transform(df_retail)
df_lrmfp['labels'] = lrmfp_kmeans_pipeline.named_steps['KMEANS'].labels_.astype(str)

summary_table = pd.concat(
    [
        df_lrmfp.groupby('labels').mean(numeric_only=True).round(1),
        df_lrmfp.groupby('labels').agg(count=('Customer ID','count'))
    ],
    axis=1)

In [43]:
summary_table

,Length,Recency,Monetary,Frequency,Periodicity,count
labels,,,,,,
0,24.6,61.6,74.7,1.9,3.4,1394
1,216.5,161.2,69.8,4.0,44.1,1180
2,329.3,75.2,66.3,12.7,38.4,815
3,21.3,264.1,83.5,1.5,2.4,925


```
A partir de la tabla de resumen por clúster, es posible notar algunas diferencias entre cada uno, denotando distintos tipos de clientes. A primera vista, los 4 clústers no parecen tener grandes diferencias entre la cantidad de clientes en cada uno.
- Para el clúster 0, se tienen posiblemente a los clientes menos fieles, con poca frecuencia de visitas a la empresa en pequeños intervalos de tiempo. Mantienen un gasto mayor que los clústers 0 y 2, pero con una baja tendencia a repetirlos.
- Para el clúster 1, se tienen clientes con alta tendencia de repetir sus compras, en comparación a los clústers 2 y 3, con baja periodicidad y visitas a la empresa, pero mostrando fidelidad a esta.
- Para el clúster 2, se tienen a los clientes más fieles, mostrando una alta frecuencia de visitas a la empresa, a pesar de su periodicidad más alta y baja tendencia a repetir sus compras, manteniendo un bajo gasto. 
- Para el clúster 3, se tienen clientes que gastaron más en promedio que en el resto de los clústers en un corto período de tiempo. Es probable que repitan su compra pero no muestran tanta fidelidad.
```

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [44]:
labels = lrmfp_kmeans_pipeline.named_steps['KMEANS'].labels_.astype(str)

fig = px.scatter(
    df_LRMPF, x=X_tsne[:,0], y=X_tsne[:,1], color=labels, title='Visualización de atributos LRMPF con t-SNE',
    hover_name='Customer ID', hover_data=['Length','Recency','Frequency','Monetary','Periodicity']
)

fig.show()

```
Existen sectores en que los clústers son bien diferenciados pero también hay una menor cantidad en la que no. Por ejemplo, los clústers 1 y 2 parecieran esta superpuestos en algunos puntos, al igual que los clústers 0 y 3 en la parte inferior. También existen pequeñas agrupaciones (parte superior del gráfico del clúster 0) alejadas de donde se concentran más los puntos, y estos son indicados como del mismo clúster, siendo que posiblemente podrían ser clústers distintos. Los clústers 0, 1 y 2 parecieran estar bien separados a pesar de los pocos puntos sobrepuestos, sin embargo, existe otra agrupación que parece envolver la aglomeración del centro y que corresponde también al clústers 0, siendo que gráficamente podrían parecer a otro clúster. Finalmente, es posible que una mayor cantidad de clústers o utilizar DBSCAN podría seguir más con la intuición que se tiene gráficamente y quizá dar resultados más interpretables.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>